In [1]:
pip install pyzbar opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install paho-mqtt

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyzbar import pyzbar
import paho.mqtt.client as mqtt
import cv2

In [4]:
#Informacion imagen
class InfoImg:
    def __init__(self, tipo, datos):
        self.tipo = tipo
        self.datos = datos

In [ ]:
def decode(image):
    # Decodes all barcodes from an image
    decoded_objects = pyzbar.decode(image)

    if not decoded_objects:
        print("No barcodes detected.")
        return None  # Retorna None si no hay códigos de barras

    # Procesar solo el primer código de barras detectado
    obj = decoded_objects[0]
    print("Detected barcode:", obj)
    image = draw_barcode(obj, image)

    # Decodificar el tipo y los datos
    tipo = obj.type
    datos = obj.data.decode("utf-8")  # Decodificar los datos a texto
    print("Type:", tipo)
    print("Data:", datos)
    print()

    # Retornar una instancia de InfoImg
    return InfoImg(tipo, datos)


def draw_barcode(decoded, image):
    # n_points = len(decoded.polygon)
    # for i in range(n_points):
    #     image = cv2.line(image, decoded.polygon[i], decoded.polygon[(i+1) % n_points], color=(0, 255, 0), thickness=5)
    # uncomment above and comment below if you want to draw a polygon and not a rectangle
    image = cv2.rectangle(image, (decoded.rect.left, decoded.rect.top), 
                            (decoded.rect.left + decoded.rect.width, decoded.rect.top + decoded.rect.height),
                            color=(0, 255, 0),
                            thickness=5)
    return image

def publicador(infoImg, numeroGrupo):
    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)  # Actualizar API
    client.connect("test.mosquitto.org", 1883, 60)
    
    if str(infoImg.datos)[0] == str(numeroGrupo):
        siguiente_grupo = str(infoImg.datos)[1]
        nuevo_dato = int(infoImg.datos) + 10
        client.publish(f"PATRONES2025/G{siguiente_grupo}", nuevo_dato)
        print(f"Enviando {nuevo_dato} al grupo {siguiente_grupo}")
        client.disconnect()  # Desconectar después de enviar
        return

def publicador(infoImg, numeroGrupo):
    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)  # Usar API V2
    client.connect("test.mosquitto.org", 1883, 60)

    # Caso 1: Grupo inicial (3)
    if str(infoImg.datos)[0] == str(numeroGrupo):
        siguiente_grupo = str(infoImg.datos)[1]  # Obtener el siguiente dígito (1)
        nuevo_dato = int(infoImg.datos) + 10  # 312546789760 → 312546789770
        client.publish(f"PATRONES2025/#", "Del grupo 1 al grupo "+ siguiente_grupo)
        client.publish(f"PATRONES2025/G{siguiente_grupo}", nuevo_dato)
        print(f"Enviando {nuevo_dato} al grupo {siguiente_grupo}")
        client.disconnect()

        # Llamada recursiva para el siguiente grupo (1)
        publicador(infoImg, int(siguiente_grupo))  # Ahora procesa el grupo 1

    else:
        # Caso 2: Grupo intermedio (1)
        def on_message(client, userdata, message):
            print(f"Mensaje recibido: {message.payload.decode()}")
            nuevo_dato = int(message.payload.decode()) + 10
            datos_str = str(infoImg.datos)

            if str(numeroGrupo) not in datos_str:
                print("Grupo no está en los datos")
                client.disconnect()
                return

            index_actual = datos_str.index(str(numeroGrupo))
            if index_actual + 1 >= len(datos_str):
                print("Fin de la secuencia")
                client.disconnect()
                return

            siguiente_grupo = datos_str[index_actual + 1]
            client.publish(f"PATRONES2025/G{siguiente_grupo}", nuevo_dato)
            #client.publish(f"PATRONES2025/#", "GRUPO 1 a GRUPO" + siguiente_grupo)
            print(f"Publicado a G{siguiente_grupo}: {nuevo_dato}")
            client.disconnect()

        client.on_message = on_message
        client.subscribe(f"PATRONES2025/G{numeroGrupo}")
        print(f"Escuchando en PATRONES2025/G{numeroGrupo}...")
        client.loop_forever()

In [27]:
if __name__ == "__main__":
    from glob import glob

    barcodes = glob("Caso5.png")

    for barcode_file in barcodes:
        # Cargar la imagen con OpenCV
        img = cv2.imread(barcode_file)

        # Decodificar el primer código de barras detectado
        decoded_info = decode(img)

        if decoded_info is not None:
            print(f"Tipo: {decoded_info.tipo}, Datos: {decoded_info.datos}")
            # Intentar convertir los datos a un número
            try:
                datos = int(decoded_info.datos)  # Convertir los datos a un número entero si es posible
                publicador(decoded_info, 1)

            except ValueError:
                print("No contiene datos, terminado programa")
                pass  # Si no es un número, mantener los datos como texto     
        else:
            print("No se detectaron códigos de barras.")

    

Detected barcode: Decoded(data=b'312546789760', type='DATABAR_EXP', rect=Rect(left=17, top=8, width=650, height=111), polygon=[Point(x=17, y=9), Point(x=17, y=119), Point(x=262, y=119), Point(x=507, y=118), Point(x=667, y=116), Point(x=667, y=8)], quality=141, orientation='UP')
Type: DATABAR_EXP
Data: 312546789760

Tipo: DATABAR_EXP, Datos: 312546789760
Escuchando en PATRONES2025/G1...
Mensaje recibido: 312546789770
Publicado a G2: 312546789780
